***Cell 1: Setup - Imports and API Configuration***

In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output

# Configure API Key
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
if not google_api_key:
    raise ValueError("Google API key not found in .env file.")
genai.configure(api_key=google_api_key)
print("✅ Google Gemini client configured successfully.")

✅ Google Gemini client configured successfully.


***Cell 2: The Control Panel (Interactive Widgets)***

In [5]:
# --- Section 1: Model & Creativity Parameters ---
title_model = widgets.HTML("<h3>1. Model & Creativity Settings</h3>")

model_selector = widgets.Dropdown(
    options=['gemini-2.5-flash', 'gemini-1.5-flash-latest', 'gemini-1.5-pro'],
    value='gemini-1.5-flash-latest',
    description='Model:',
    layout={'width': 'auto'}
)
temperature_slider = widgets.FloatSlider(
    value=0.7, min=0, max=2.0, step=0.1, description='Temperature:', readout_format='.1f', layout={'width': 'auto'}
)
top_p_slider = widgets.FloatSlider(
    value=0.95, min=0, max=1.0, step=0.01, description='Top-P:', readout_format='.2f', layout={'width': 'auto'}
)
top_k_slider = widgets.IntSlider(
    value=40, min=1, max=100, step=1, description='Top-K:', layout={'width': 'auto'}
)
# Tip about using creativity parameters
param_tip = widgets.HTML("<p><i><b>Pro Tip:</b> For best results, adjust either <b>Temperature</b> OR <b>Top-P/Top-K</b>, not all at once.</i></p>")

# We use an HBox to arrange these widgets horizontally for a cleaner look
model_creativity_box = widgets.HBox([model_selector, temperature_slider, top_p_slider, top_k_slider])


# --- Section 2: Output Control ---
title_output = widgets.HTML("<h3>2. Output Control</h3>")
max_tokens_slider = widgets.IntSlider(
    value=2048, min=256, max=8192, step=256, description='Max Tokens:'
)

# --- Section 3: Prompt Definition ---
title_prompts = widgets.HTML("<h3>3. Prompt Definition</h3>")
system_prompt_input = widgets.Textarea(
    value='You are a helpful and creative AI assistant.',
    description='System Prompt:', layout={'height': '100px', 'width': '99%'}
)
user_prompt_input = widgets.Textarea(
    value="Explain the concept of 'prompt engineering' to a junior developer, providing a simple, real-world analogy.",
    description='User Prompt:', layout={'height': '200px', 'width': '99%'}
)

# --- Section 4: Execution & Output ---
title_execution = widgets.HTML("<h3>4. Execution</h3>")
run_button = widgets.Button(description='Generate Response', button_style='success', icon='rocket', layout={'width': '200px'})
output_area = widgets.Output()


# We use a VBox to arrange all sections vertically
# This creates the final, clean layout of our entire application
control_panel = widgets.VBox([
    title_model,
    model_creativity_box,
    param_tip,
    title_output,
    max_tokens_slider,
    title_prompts,
    system_prompt_input,
    user_prompt_input,
    title_execution,
    run_button,
    output_area
])

display(control_panel)

***Cell 3: The Execution Logic (The "Engine")***

In [6]:
def generate_response_with_streaming(b):
    """Triggers on button click, calls Gemini API with all configured parameters and streaming."""
    with output_area:
        clear_output(wait=True) # wait=True prevents flickering
        print("🚀 Sending request to Gemini...")

        # Gather all values from the widgets
        generation_config = {
            "temperature": temperature_slider.value,
            "top_p": top_p_slider.value,
            "top_k": top_k_slider.value,
            "max_output_tokens": max_tokens_slider.value,
        }
        
        try:
            model = genai.GenerativeModel(
                model_selector.value,
                system_instruction=system_prompt_input.value
            )

            # Generate content with stream=True and the full generation_config
            response_stream = model.generate_content(
                user_prompt_input.value,
                stream=True,
                generation_config=generation_config
            )
            
            # --- Streaming and Enhanced Output Display ---
            clear_output(wait=True)
            
            # A styled container for the response
            html_output = widgets.HTML(value="<div style='border: 1px solid #e0e0e0; border-radius: 5px; padding: 15px; background-color: #f9f9f9;'></div>")
            display(html_output)
            
            full_response_text = ""
            for chunk in response_stream:
                if chunk.text:
                    full_response_text += chunk.text
                    html_output.value = f"<div style='border: 1px solid #e0e0e0; border-radius: 5px; padding: 15px; background-color: #f9f9f9;'>{full_response_text.replace('\n', '<br>')}</div>"
            
            if not full_response_text.strip():
                html_output.value = "<div style='border: 1px solid #e0e0e0; border-radius: 5px; padding: 15px; background-color: #f9f9f9;'><i>O modelo não retornou nenhum conteúdo.</i></div>"

        except Exception as e:
            clear_output(wait=True)
            display(widgets.HTML(value=f"<div style='border: 1px solid red; padding: 15px; color: red;'><b>An error occurred:</b><br>{e}</div>"))

***Cell 4: Connecting the Button to the Logic***

In [7]:
# This line "connects" the button to our function.
# When the button is clicked, the 'generate_response' function will be executed.
run_button.on_click(generate_response_with_streaming)